## keras 모델 불러오기

In [1]:
import glob
from keras.models import load_model
import numpy as np
import pandas as pd

Using TensorFlow backend.


In [2]:
model_files = glob.glob('weights.*.hdf5')
# model_files

In [3]:
last_model = sorted(model_files)[-1]
last_model

'weights.005-0.9497.hdf5'

In [4]:
from konlpy.tag import Twitter
twitter = Twitter()

In [5]:
def make_vector(string_list, max_length=100):
    x_list = []
    for keyword in string_list:
        if keyword in read_dictionary:
            x_list.append(read_dictionary[keyword])
        else:
            x_list.append(0)    
    return np.array(x_list[:max_length] + [0]*(max_length-len(x_list)))

def merge_result(y_new):
    result = pd.DataFrame(y_new[0]*100, columns=['분류비율'])
    result['분류'] = pd.read_csv('./paper_class', header=None)

    result = result.sort_values(by=['분류비율'], ascending=False)
    result['분류비율'] = result['분류비율'].map('{:,.2f} %'.format)
    return result

### 모델 불러오기 및 단어사전 만들기

In [6]:
model = load_model(last_model)

In [7]:
read_dictionary = np.load('vocabulary_dict.npy').item()

In [8]:
# subject = "1∼3분기 대비 4분기 매출액 변화가 이익조정에 미치는 영향" #경제경영 > 경영학
# subject = "Investigating the Determinants of Major IT Incident Tickets:" #경제경영 > 경영학
# subject = "의사결정나무와 손실함수를 이용한 공정파라미터 허용차 설계에 관한 연구" #경제경영 > 경영학
# subject = "보육교사가 지각하는 원장의 감성리더십이 교사직 지속이유와 이직이유에 미치는 영향" # 사회과학 > 사회복지학
subject = "데이터 변형성 기반 유사성 연결을 위한 단어 추천 알고리즘" #  공학 > 전자/정보통신공학
# subject = "균형능력 운동치료 시스템을 이용한 치료가 급성기 뇌졸중 환자의 균형과 이동능력에 미치는 영향" # 의약학 > 의학일반
# subject = "사회적 지지 프로그램이 재가 치매노인 돌봄 제공자의 부담감과 안녕에 미치는 효과"# 의약학 > 의학일반

### 검증 제목 입력

In [9]:
subject = "데이터 변형성 기반 유사성 연결을 위한 단어 추천 알고리즘" #  공학 > 전자/정보통신공학

In [10]:
p = twitter.pos(subject)
p = [value for value, key in p]
x_new=make_vector(p)
x_new=x_new.reshape(-1, 100)

In [11]:
# 기존 모델에 적용
y_new = model.predict(x_new)

In [12]:
result = merge_result(y_new)  # 결과 정리
result[:3]

,분류비율,분류
7,61.27 %,공학
2,31.82 %,사회과학
3,1.99 %,경제경영


### 여러 구문 입력

In [13]:
subjects = ["1∼3분기 대비 4분기 매출액 변화가 이익조정에 미치는 영향",                             #경제경영
            "Investigating the Determinants of Major IT Incident Tickets:",                         #경제경영
            "의사결정나무와 손실함수를 이용한 공정파라미터 허용차 설계에 관한 연구",                #경제경영
            "보육교사가 지각하는 원장의 감성리더십이 교사직 지속이유와 이직이유에 미치는 영향",     # 사회과학
            "데이터 변형성 기반 유사성 연결을 위한 단어 추천 알고리즘",                             #  공학
            "균형능력 운동치료 시스템을 이용한 치료가 급성기 뇌졸중 환자의 균형과 이동능력에 미치는 영향", # 의약학
            "사회적 지지 프로그램이 재가 치매노인 돌봄 제공자의 부담감과 안녕에 미치는 효과" ]# 의약학

In [14]:
for subject in subjects:
    p = twitter.pos(subject)
    p = [value for value, key in p]
    x_new=make_vector(p)
    x_new=x_new.reshape(-1, 100)
  
    y_new = model.predict(x_new)  # 기존 모델에 적용
    result = merge_result(y_new)  # 결과 정리
    print(result[:3])

      분류비율    분류
3  98.99 %  경제경영
2   0.73 %  사회과학
0   0.13 %   인문학
      분류비율    분류
7  68.30 %    공학
3  21.57 %  경제경영
2   7.94 %  사회과학
      분류비율    분류
7  46.10 %    공학
9  32.93 %  농수해양
3   8.64 %  경제경영
      분류비율    분류
2  85.88 %  사회과학
6   4.65 %  자연과학
5   3.59 %  예술체육
      분류비율    분류
7  61.27 %    공학
2  31.82 %  사회과학
3   1.99 %  경제경영
      분류비율    분류
8  81.56 %   의약학
2  17.16 %  사회과학
3   0.45 %  경제경영
      분류비율    분류
8  66.51 %   의약학
2  32.87 %  사회과학
9   0.23 %  농수해양
